# AG2 (AutoGen) Example

This is an example using the AG2 version of AutoGen (which is distinct from the Microsoft version).

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

Create the model client:

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

AutoGen uses the TextMessage class for messages to the LLM:

In [ ]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="I'd like to go to London", source="user")
message

Set up a small tool using the Serper web search from LangChain:

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()

def get_city_price(city: str) -> str:
    """A search tool using Serper API to find airline ticket prices to specific cities.

    Args:
        city: The city to search for.
    """
    search_query = f"Airline prices for travel to {city}"
    return serper.run(search_query)

Set up an agent with access to the tool:

In [ ]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client_stream=True,
    tools=[get_city_price],
    reflect_on_tool_use=True
)

Run the agent:

In [ ]:
from autogen_core import CancellationToken

response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content